## Extraction des variables température, salinité et force du vent sur la zone Med est

### 1. Extraction des variables TEMP et SAL

#### Extraction 24h en amont

In [1]:
from tqdm import tqdm
import geopandas as gpd
import xarray as xr
import rioxarray
import numpy as np
from shapely.geometry import Point, box
import glob
import os
import re
from datetime import datetime, timedelta


def get_netcdf_paths_for_period(dt, base_folder, hours=24):
    start_dt = dt - timedelta(hours=hours)
    files = []

    for year in range(start_dt.year, dt.year + 1):
        year_folder = os.path.join(base_folder, str(year))
        pattern = os.path.join(year_folder, "MARC_F2-MARS3D-MENOR1200_????????T????Z.nc")
        candidates = glob.glob(pattern)

        def extract_datetime_from_filename(f):
            match = re.search(r"_(\d{8}T\d{4})Z\.nc$", f)
            if not match:
                return None
            return datetime.strptime(match.group(1), "%Y%m%dT%H%M")

        for f in candidates:
            f_dt = extract_datetime_from_filename(f)
            if f_dt and start_dt <= f_dt <= dt:
                files.append((f, f_dt))

    files.sort(key=lambda x: x[1])
    if not files:
        raise FileNotFoundError(f"Aucun fichier trouvé entre {start_dt} et {dt}")

    return [f for f, _ in files]


def get_temp_sal_for_poly(poly, depth_sampling, ncdf_path):
    ds = xr.open_dataset(ncdf_path, engine="netcdf4")
    if not hasattr(ds, 'crs'):
        ds = ds.rio.write_crs("EPSG:4326")

    bathy = ds['H0']
    temp = ds['TEMP']
    sal = ds['SAL']

    poly_gs = gpd.GeoSeries([poly], crs="EPSG:4326")
    poly_proj = poly_gs.to_crs(bathy.rio.crs).iloc[0]

    transform = bathy.rio.transform()
    height, width = bathy.shape

    coords = []
    bathy_vals = []

    for j in range(height):
        for i in range(width):
            x_min, y_max = transform * (i, j)
            x_max, y_min = transform * (i + 1, j + 1)
            pixel_poly = box(x_min, y_min, x_max, y_max)
            if poly_proj.intersects(pixel_poly):
                b_val = bathy.values[j, i]
                if not np.isnan(b_val) and b_val > 0:
                    coords.append((j, i))
                    bathy_vals.append(b_val)

    if not bathy_vals:
        pixel_centers = []
        for j in range(height):
            for i in range(width):
                x_c, y_c = transform * (i + 0.5, j + 0.5)
                pixel_centers.append((j, i, Point(x_c, y_c)))

        distances = []
        for (j, i, pt) in pixel_centers:
            dist = pt.distance(poly_proj)
            b_val = bathy.values[j, i]
            if not np.isnan(b_val) and b_val > 0:
                distances.append((dist, j, i, b_val))

        distances.sort(key=lambda x: x[0])
        closest = distances[:3]
        coords = [(j, i) for _, j, i, _ in closest]
        bathy_vals = [b for _, _, _, b in closest]

    layers_phys = [int(depth_sampling / b * 60) for b in bathy_vals]
    layers_phys = [60 - l for l in layers_phys]
    layers_index = [max(0, min(l - 1, 59)) for l in layers_phys]

    temp_values = []
    sal_values = []
    for (j, i), l in zip(coords, layers_index):
        t_val = temp.values[0, l, j, i]
        s_val = sal.values[0, l, j, i]
        if not np.isnan(t_val):
            temp_values.append(t_val)
        if not np.isnan(s_val):
            sal_values.append(s_val)

    ds.close()

    return temp_values, sal_values


def main():
    base_folder = "/home/paulinev/Bureau/Marbec_data/BiodivMed/MARS3D/Med_MENOR/Aggregated/SAL-TEMP_latlon/3H/"
    gdf = gpd.read_file("adne_extract_med_ouest.geojson")

    min_temp = []
    max_temp = []
    mean_temp = []

    min_sal = []
    max_sal = []
    mean_sal = []

    for idx, row in tqdm(gdf.iterrows(), total=len(gdf), desc="Extraction Temp & Sal sur 24h"):
        dt = row['datetime']
        try:
            files = get_netcdf_paths_for_period(dt, base_folder, hours=24)
            all_temp_values = []
            all_sal_values = []

            for f in files:
                t_vals, s_vals = get_temp_sal_for_poly(row['geometry'], row['depth_sampling'], f)
                all_temp_values.extend(t_vals)
                all_sal_values.extend(s_vals)

            if all_temp_values:
                min_temp.append(np.min(all_temp_values))
                max_temp.append(np.max(all_temp_values))
                mean_temp.append(np.mean(all_temp_values))
            else:
                min_temp.append(np.nan)
                max_temp.append(np.nan)
                mean_temp.append(np.nan)

            if all_sal_values:
                min_sal.append(np.min(all_sal_values))
                max_sal.append(np.max(all_sal_values))
                mean_sal.append(np.mean(all_sal_values))
            else:
                min_sal.append(np.nan)
                max_sal.append(np.nan)
                mean_sal.append(np.nan)

        except FileNotFoundError as e:
            print(f"Fichier manquant pour {dt}: {e}")
            min_temp.append(np.nan)
            max_temp.append(np.nan)
            mean_temp.append(np.nan)

            min_sal.append(np.nan)
            max_sal.append(np.nan)
            mean_sal.append(np.nan)

    gdf['temp_min_24h'] = min_temp
    gdf['temp_max_24h'] = max_temp
    gdf['temp_mean_24h'] = mean_temp

    gdf['sal_min_24h'] = min_sal
    gdf['sal_max_24h'] = max_sal
    gdf['sal_mean_24h'] = mean_sal

    gdf.to_file("adne_extract_med_ouest.geojson", driver="GeoJSON")


if __name__ == "__main__":
    main()


Extraction Temp & Sal sur 24h: 100%|██████████| 70/70 [1:44:36<00:00, 89.66s/it]


In [ ]:
# 7 jours 

from tqdm import tqdm
import geopandas as gpd
import xarray as xr
import rioxarray
import numpy as np
from shapely.geometry import Point, box
import os
from datetime import timedelta


def get_netcdf_paths_for_period(dt, base_folder, days, stat_type):
    """Retourne la liste des fichiers journaliers pour les `days` jours avant dt (inclus)."""
    start_dt = dt - timedelta(days=days)
    files = []

    for day in (start_dt + timedelta(n) for n in range((dt - start_dt).days + 1)):
        year_folder = os.path.join(base_folder, str(day.year))
        fname = f"MARS3D_{day.strftime('%Y%m%d')}_{stat_type}.nc"
        fpath = os.path.join(year_folder, fname)
        if os.path.exists(fpath):
            files.append(fpath)

    if not files:
        raise FileNotFoundError(
            f"Aucun fichier {stat_type} trouvé entre {start_dt} et {dt}"
        )

    return files



def get_temp_sal_for_poly(poly, depth_sampling, ncdf_path):
    ds = xr.open_dataset(ncdf_path, engine="netcdf4")
    if not hasattr(ds, 'crs'):
        ds = ds.rio.write_crs("EPSG:4326")

    bathy = ds['H0']
    temp = ds['TEMP']
    sal = ds['SAL']

    poly_gs = gpd.GeoSeries([poly], crs="EPSG:4326")
    poly_proj = poly_gs.to_crs(bathy.rio.crs).iloc[0]

    transform = bathy.rio.transform()
    height, width = bathy.shape

    coords = []
    bathy_vals = []

    for j in range(height):
        for i in range(width):
            x_min, y_max = transform * (i, j)
            x_max, y_min = transform * (i + 1, j + 1)
            pixel_poly = box(x_min, y_min, x_max, y_max)
            if poly_proj.intersects(pixel_poly):
                b_val = bathy.values[j, i]
                if not np.isnan(b_val) and b_val > 0:
                    coords.append((j, i))
                    bathy_vals.append(b_val)

    if not bathy_vals:
        pixel_centers = []
        for j in range(height):
            for i in range(width):
                x_c, y_c = transform * (i + 0.5, j + 0.5)
                pixel_centers.append((j, i, Point(x_c, y_c)))

        distances = []
        for (j, i, pt) in pixel_centers:
            dist = pt.distance(poly_proj)
            b_val = bathy.values[j, i]
            if not np.isnan(b_val) and b_val > 0:
                distances.append((dist, j, i, b_val))

        distances.sort(key=lambda x: x[0])
        closest = distances[:3]
        coords = [(j, i) for _, j, i, _ in closest]
        bathy_vals = [b for _, _, _, b in closest]

    layers_phys = [int(depth_sampling / b * 60) for b in bathy_vals]
    layers_phys = [60 - l for l in layers_phys]
    layers_index = [max(0, min(l - 1, 59)) for l in layers_phys]

    temp_values = []
    sal_values = []
    for (j, i), l in zip(coords, layers_index):
        if temp.ndim == 4:  # Ancien format avec time
            t_val = temp.values[0, l, j, i]
            s_val = sal.values[0, l, j, i]
        else:  # Nouveau format journalier sans time
            t_val = temp.values[l, j, i]
            s_val = sal.values[l, j, i]
        if not np.isnan(t_val):
            temp_values.append(t_val)
        if not np.isnan(s_val):
            sal_values.append(s_val)


    ds.close()

    return temp_values, sal_values


def main():
    base_folder = "/home/paulinev/Bureau/Marbec_data/BiodivMed/MARS3D/Med_MENOR/Aggregated/SAL-TEMP_latlon/Daily/Med-Ouest"
    gdf = gpd.read_file("sal_temp/adne_extract_med_ouest.geojson")

    temp_max7 = []
    temp_min7 = []
    temp_mean7 = []
    sal_max7 = []
    sal_min7 = []
    sal_mean7 = []

    for idx, row in tqdm(gdf.iterrows(), total=len(gdf), desc="Extraction 7 jours"):
        dt = row['date']

        try:
            # MAX du max
            files_max = get_netcdf_paths_for_period(dt, base_folder, days=7, stat_type="max")
            all_temp, all_sal = [], []
            for f in files_max:
                t_vals, s_vals = get_temp_sal_for_poly(row['geometry'], row['depth_sampling'], f)
                all_temp.extend(t_vals)
                all_sal.extend(s_vals)
            temp_max7.append(np.nanmax(all_temp) if all_temp else np.nan)
            sal_max7.append(np.nanmax(all_sal) if all_sal else np.nan)

            # MIN du min
            files_min = get_netcdf_paths_for_period(dt, base_folder, days=7, stat_type="min")
            all_temp, all_sal = [], []
            for f in files_min:
                t_vals, s_vals = get_temp_sal_for_poly(row['geometry'], row['depth_sampling'], f)
                all_temp.extend(t_vals)
                all_sal.extend(s_vals)
            temp_min7.append(np.nanmin(all_temp) if all_temp else np.nan)
            sal_min7.append(np.nanmin(all_sal) if all_sal else np.nan)

            # MOYENNE des mean
            files_mean = get_netcdf_paths_for_period(dt, base_folder, days=7, stat_type="mean")
            all_temp, all_sal = [], []
            for f in files_mean:
                t_vals, s_vals = get_temp_sal_for_poly(row['geometry'], row['depth_sampling'], f)
                all_temp.extend(t_vals)
                all_sal.extend(s_vals)
            temp_mean7.append(np.nanmean(all_temp) if all_temp else np.nan)
            sal_mean7.append(np.nanmean(all_sal) if all_sal else np.nan)

        except FileNotFoundError as e:
            print(f"Fichier manquant pour {dt}: {e}")
            temp_max7.append(np.nan)
            temp_min7.append(np.nan)
            temp_mean7.append(np.nan)
            sal_max7.append(np.nan)
            sal_min7.append(np.nan)
            sal_mean7.append(np.nan)

    gdf['temp_max_7j'] = temp_max7
    gdf['temp_min_7j'] = temp_min7
    gdf['temp_mean_7j'] = temp_mean7
    gdf['sal_max_7j'] = sal_max7
    gdf['sal_min_7j'] = sal_min7
    gdf['sal_mean_7j'] = sal_mean7

    gdf.to_file("sal_temp/adne_extract_med_ouest.geojson", driver="GeoJSON")


if __name__ == "__main__":
    main()


Extraction 7 jours: 100%|██████████| 70/70 [00:00<00:00, 5875.67it/s]

Fichier manquant pour 2020-07-16 00:00:00: Aucun fichier max trouvé entre 2020-07-09 00:00:00 et 2020-07-16 00:00:00
Fichier manquant pour 2018-04-05 00:00:00: Aucun fichier max trouvé entre 2018-03-29 00:00:00 et 2018-04-05 00:00:00
Fichier manquant pour 2018-04-05 00:00:00: Aucun fichier max trouvé entre 2018-03-29 00:00:00 et 2018-04-05 00:00:00
Fichier manquant pour 2018-04-06 00:00:00: Aucun fichier max trouvé entre 2018-03-30 00:00:00 et 2018-04-06 00:00:00
Fichier manquant pour 2019-07-11 00:00:00: Aucun fichier max trouvé entre 2019-07-04 00:00:00 et 2019-07-11 00:00:00
Fichier manquant pour 2020-05-20 00:00:00: Aucun fichier max trouvé entre 2020-05-13 00:00:00 et 2020-05-20 00:00:00
Fichier manquant pour 2020-05-20 00:00:00: Aucun fichier max trouvé entre 2020-05-13 00:00:00 et 2020-05-20 00:00:00
Fichier manquant pour 2020-07-16 00:00:00: Aucun fichier max trouvé entre 2020-07-09 00:00:00 et 2020-07-16 00:00:00
Fichier manquant pour 2020-05-28 00:00:00: Aucun fichier max tro

In [2]:
from tqdm import tqdm
import geopandas as gpd
import xarray as xr
import rioxarray
import numpy as np
from shapely.geometry import Point, box
import os
from datetime import timedelta
from dateutil.relativedelta import relativedelta


def get_netcdf_paths_for_last_month(dt, base_folder, stat_type):
    """
    Retourne la liste des fichiers journaliers pour un mois glissant
    allant de (dt - 1 mois) à dt inclus.
    """
    start_dt = dt - relativedelta(months=1)
    end_dt = dt

    files = []
    for day in (start_dt + timedelta(n) for n in range((end_dt - start_dt).days + 1)):
        year_folder = os.path.join(base_folder, str(day.year))
        fname = f"MARS3D_{day.strftime('%Y%m%d')}_{stat_type}.nc"
        fpath = os.path.join(year_folder, fname)
        if os.path.exists(fpath):
            files.append(fpath)

    if not files:
        raise FileNotFoundError(
            f"Aucun fichier {stat_type} trouvé entre {start_dt} et {end_dt}"
        )

    return files


def get_temp_sal_for_poly(poly, depth_sampling, ncdf_path):
    ds = xr.open_dataset(ncdf_path, engine="netcdf4")
    if not hasattr(ds, 'crs'):
        ds = ds.rio.write_crs("EPSG:4326")

    bathy = ds['H0']
    temp = ds['TEMP']
    sal = ds['SAL']

    poly_gs = gpd.GeoSeries([poly], crs="EPSG:4326")
    poly_proj = poly_gs.to_crs(bathy.rio.crs).iloc[0]

    transform = bathy.rio.transform()
    height, width = bathy.shape

    coords = []
    bathy_vals = []

    for j in range(height):
        for i in range(width):
            x_min, y_max = transform * (i, j)
            x_max, y_min = transform * (i + 1, j + 1)
            pixel_poly = box(x_min, y_min, x_max, y_max)
            if poly_proj.intersects(pixel_poly):
                b_val = bathy.values[j, i]
                if not np.isnan(b_val) and b_val > 0:
                    coords.append((j, i))
                    bathy_vals.append(b_val)

    if not bathy_vals:
        pixel_centers = []
        for j in range(height):
            for i in range(width):
                x_c, y_c = transform * (i + 0.5, j + 0.5)
                pixel_centers.append((j, i, Point(x_c, y_c)))

        distances = []
        for (j, i, pt) in pixel_centers:
            dist = pt.distance(poly_proj)
            b_val = bathy.values[j, i]
            if not np.isnan(b_val) and b_val > 0:
                distances.append((dist, j, i, b_val))

        distances.sort(key=lambda x: x[0])
        closest = distances[:3]
        coords = [(j, i) for _, j, i, _ in closest]
        bathy_vals = [b for _, _, _, b in closest]

    layers_phys = [int(depth_sampling / b * 60) for b in bathy_vals]
    layers_phys = [60 - l for l in layers_phys]
    layers_index = [max(0, min(l - 1, 59)) for l in layers_phys]

    temp_values = []
    sal_values = []
    for (j, i), l in zip(coords, layers_index):
        if temp.ndim == 4:  # Ancien format avec time
            t_val = temp.values[0, l, j, i]
            s_val = sal.values[0, l, j, i]
        else:  # Nouveau format journalier sans time
            t_val = temp.values[l, j, i]
            s_val = sal.values[l, j, i]
        if not np.isnan(t_val):
            temp_values.append(t_val)
        if not np.isnan(s_val):
            sal_values.append(s_val)

    ds.close()
    return temp_values, sal_values


def main():
    base_folder = "/home/paulinev/Bureau/Marbec_data/BiodivMed/MARS3D/Med_MENOR/Aggregated/SAL-TEMP_latlon/Daily/Med-Ouest"
    gdf = gpd.read_file("adne_extract_med_ouest.geojson")

    temp_max, temp_min, temp_mean = [], [], []
    sal_max, sal_min, sal_mean = [], [], []

    for idx, row in tqdm(gdf.iterrows(), total=len(gdf), desc="Extraction sur 1 mois glissant"):
        dt = row['date']

        try:
            # MAX du max
            files_max = get_netcdf_paths_for_last_month(dt, base_folder, stat_type="max")
            all_temp, all_sal = [], []
            for f in files_max:
                t_vals, s_vals = get_temp_sal_for_poly(row['geometry'], row['depth_sampling'], f)
                all_temp.extend(t_vals)
                all_sal.extend(s_vals)
            temp_max.append(np.nanmax(all_temp) if all_temp else np.nan)
            sal_max.append(np.nanmax(all_sal) if all_sal else np.nan)

            # MIN du min
            files_min = get_netcdf_paths_for_last_month(dt, base_folder, stat_type="min")
            all_temp, all_sal = [], []
            for f in files_min:
                t_vals, s_vals = get_temp_sal_for_poly(row['geometry'], row['depth_sampling'], f)
                all_temp.extend(t_vals)
                all_sal.extend(s_vals)
            temp_min.append(np.nanmin(all_temp) if all_temp else np.nan)
            sal_min.append(np.nanmin(all_sal) if all_sal else np.nan)

            # MOYENNE des mean
            files_mean = get_netcdf_paths_for_last_month(dt, base_folder, stat_type="mean")
            all_temp, all_sal = [], []
            for f in files_mean:
                t_vals, s_vals = get_temp_sal_for_poly(row['geometry'], row['depth_sampling'], f)
                all_temp.extend(t_vals)
                all_sal.extend(s_vals)
            temp_mean.append(np.nanmean(all_temp) if all_temp else np.nan)
            sal_mean.append(np.nanmean(all_sal) if all_sal else np.nan)

        except FileNotFoundError as e:
            print(f"Fichier manquant pour {dt}: {e}")
            temp_max.append(np.nan)
            temp_min.append(np.nan)
            temp_mean.append(np.nan)
            sal_max.append(np.nan)
            sal_min.append(np.nan)
            sal_mean.append(np.nan)

    gdf['temp_max_1m'] = temp_max
    gdf['temp_min_1m'] = temp_min
    gdf['temp_mean_1m'] = temp_mean
    gdf['sal_max_1m'] = sal_max
    gdf['sal_min_1m'] = sal_min
    gdf['sal_mean_1m'] = sal_mean

    gdf.to_file("adne_extract_med_ouest.geojson", driver="GeoJSON")


if __name__ == "__main__":
    main()


Extraction sur 1 mois glissant: 100%|██████████| 70/70 [24:25<00:00, 20.93s/it]


In [3]:
from tqdm import tqdm
import geopandas as gpd
import xarray as xr
import rioxarray
import numpy as np
from shapely.geometry import Point, box
import os
from datetime import timedelta
import pandas as pd

# -----------------------------
# Fonctions utilitaires
# -----------------------------

def get_monthly_paths(dt_start, dt_end, base_folder, stat_type):
    """Retourne les fichiers mensuels (min/max/mean) entre dt_start et le mois précédent dt_end."""
    files = []
    months = pd.date_range(start=dt_start, end=dt_end, freq='MS')  # Month Start
    for month in months[:-1]:  # tous les mois sauf le dernier
        year_folder = os.path.join(base_folder, str(month.year))
        fname = f"MARS3D_{month.strftime('%Y%m')}_{stat_type}.nc"
        fpath = os.path.join(year_folder, fname)
        if os.path.exists(fpath):
            files.append(fpath)
    return files

def get_daily_paths(dt_start, dt_end, base_folder, stat_type):
    """Retourne les fichiers journaliers (min/max/mean) entre dt_start et dt_end."""
    files = []
    for day in (dt_start + timedelta(n) for n in range((dt_end - dt_start).days + 1)):
        year_folder = os.path.join(base_folder, str(day.year))
        fname = f"MARS3D_{day.strftime('%Y%m%d')}_{stat_type}.nc"
        fpath = os.path.join(year_folder, fname)
        if os.path.exists(fpath):
            files.append(fpath)
    return files

def get_temp_sal_for_poly(poly, depth_sampling, ncdf_path):
    ds = xr.open_dataset(ncdf_path, engine="netcdf4")
    if not hasattr(ds, 'crs'):
        ds = ds.rio.write_crs("EPSG:4326")

    bathy = ds['H0']
    temp = ds['TEMP']
    sal = ds['SAL']

    poly_gs = gpd.GeoSeries([poly], crs="EPSG:4326")
    poly_proj = poly_gs.to_crs(bathy.rio.crs).iloc[0]

    transform = bathy.rio.transform()
    height, width = bathy.shape

    coords = []
    bathy_vals = []

    for j in range(height):
        for i in range(width):
            x_min, y_max = transform * (i, j)
            x_max, y_min = transform * (i + 1, j + 1)
            pixel_poly = box(x_min, y_min, x_max, y_min + (y_max - y_min))
            if poly_proj.intersects(pixel_poly):
                b_val = bathy.values[j, i]
                if not np.isnan(b_val) and b_val > 0:
                    coords.append((j, i))
                    bathy_vals.append(b_val)

    if not bathy_vals:
        # Si aucun pixel intersecte, prendre les 3 pixels les plus proches
        pixel_centers = []
        for j in range(height):
            for i in range(width):
                x_c, y_c = transform * (i + 0.5, j + 0.5)
                pixel_centers.append((j, i, Point(x_c, y_c)))

        distances = []
        for (j, i, pt) in pixel_centers:
            dist = pt.distance(poly_proj)
            b_val = bathy.values[j, i]
            if not np.isnan(b_val) and b_val > 0:
                distances.append((dist, j, i, b_val))

        distances.sort(key=lambda x: x[0])
        closest = distances[:3]
        coords = [(j, i) for _, j, i, _ in closest]
        bathy_vals = [b for _, _, _, b in closest]

    layers_phys = [int(depth_sampling / b * 60) for b in bathy_vals]
    layers_phys = [60 - l for l in layers_phys]
    layers_index = [max(0, min(l - 1, 59)) for l in layers_phys]

    temp_values = []
    sal_values = []
    for (j, i), l in zip(coords, layers_index):
        if temp.ndim == 4:  # Ancien format avec time
            t_val = temp.values[0, l, j, i]
            s_val = sal.values[0, l, j, i]
        else:  # Nouveau format journalier sans time
            t_val = temp.values[l, j, i]
            s_val = sal.values[l, j, i]
        if not np.isnan(t_val):
            temp_values.append(t_val)
        if not np.isnan(s_val):
            sal_values.append(s_val)

    ds.close()
    return temp_values, sal_values

# -----------------------------
# Main
# -----------------------------

def main():
    daily_base_folder = "/home/paulinev/Bureau/Marbec_data/BiodivMed/MARS3D/Med_MENOR/Aggregated/SAL-TEMP_latlon/Daily/Med-Ouest"
    monthly_base_folder = "/home/paulinev/Bureau/Marbec_data/BiodivMed/MARS3D/Med_MENOR/Aggregated/SAL-TEMP_latlon/Monthly/Med-Ouest"
    gdf = gpd.read_file("adne_extract_med_ouest.geojson")

    temp_max, temp_min, temp_mean = [], [], []
    sal_max, sal_min, sal_mean = [], [], []

    for idx, row in tqdm(gdf.iterrows(), total=len(gdf), desc="Extraction 1 an"):
        dt = row['date']

        dt_start = dt - timedelta(days=365)
        last_month_start = dt.replace(day=1)

        try:
            all_stats = {}
            for stat in ['max', 'min', 'mean']:
                # Fichiers mensuels sauf dernier mois
                files_monthly = get_monthly_paths(dt_start, last_month_start, monthly_base_folder, stat)
                # Fichiers journaliers du dernier mois
                files_daily = get_daily_paths(last_month_start, dt, daily_base_folder, stat)
                files = files_monthly + files_daily

                all_temp, all_sal = [], []
                for f in files:
                    t_vals, s_vals = get_temp_sal_for_poly(row['geometry'], row['depth_sampling'], f)
                    all_temp.extend(t_vals)
                    all_sal.extend(s_vals)

                if stat == 'max':
                    temp_max.append(np.nanmax(all_temp) if all_temp else np.nan)
                    sal_max.append(np.nanmax(all_sal) if all_sal else np.nan)
                elif stat == 'min':
                    temp_min.append(np.nanmin(all_temp) if all_temp else np.nan)
                    sal_min.append(np.nanmin(all_sal) if all_sal else np.nan)
                elif stat == 'mean':
                    temp_mean.append(np.nanmean(all_temp) if all_temp else np.nan)
                    sal_mean.append(np.nanmean(all_sal) if all_sal else np.nan)

        except FileNotFoundError as e:
            print(f"Fichier manquant pour {dt}: {e}")
            temp_max.append(np.nan)
            temp_min.append(np.nan)
            temp_mean.append(np.nan)
            sal_max.append(np.nan)
            sal_min.append(np.nan)
            sal_mean.append(np.nan)

    gdf['temp_max_1y'] = temp_max
    gdf['temp_min_1y'] = temp_min
    gdf['temp_mean_1y'] = temp_mean
    gdf['sal_max_1y'] = sal_max
    gdf['sal_min_1y'] = sal_min
    gdf['sal_mean_1y'] = sal_mean

    gdf.to_file("adne_extract_med_ouest.geojson", driver="GeoJSON")

if __name__ == "__main__":
    main()


Extraction 1 an: 100%|██████████| 70/70 [22:49<00:00, 19.57s/it]
